In [66]:
import numpy as np
import pandas as pd
import xarray as xr
import random
import time
import os
from pathlib import Path

In [67]:
def fetch_data():
    '''
    reads the raw data
    '''
    dir_path = Path().resolve().as_posix()
    # print(dir_path)
    df = pd.read_json("https://raw.githubusercontent.com/acycliq/issplus/master/dashboard/data/img/default/json/iss.json")
    GeneExp = np.load(dir_path + '/../data_preprocessed/GeneExp.npy')
    genes = [line.rstrip("\n''") for line in open(dir_path + '/../data_preprocessed/genes.csv')]
    ctc = [line.rstrip("\n''") for line in open(dir_path + '/../data_preprocessed/cell_to_class_map.csv')]

    # Rename PC.CA2 to PC.Other1 and PC.CA3 to PC.Other2
    ctc = ['PC.Other1' if x == 'PC.CA2' else x for x in ctc]
    ctc = ['PC.Other2' if x == 'PC.CA3' else x for x in ctc]

    ge = xr.DataArray(GeneExp, coords=[genes, ctc], dims=['Genes', 'Class'])
    return df, ge

In [68]:
def best_class(df):
    '''
    Returns a list with the names of all the optimal/best classes
    '''
    class_name = df['ClassName']
    prob = df['Prob']
    out = [class_name[n][np.argmax(prob[n])] for n in range(class_name.shape[0])]
    return out

In [69]:
def draw_gene_expression(df, ge):
    '''
    Main purpose is to make gene expression 2d arrays of dimension
    num_of_genes by num_of_classes by randomly sampling from the main
    (full) expression matrix (which is of size num_of_genes by num_of_cells)
    '''
    # cell class (unique and ranked alphabetically)
    best_classes = sorted(set(df['best_class']))
    class_list = ge.Class.values.tolist()
    N = len(best_classes)
    M = ge.shape[0]
    out = {'cid': [],
           'Cell_Num': [],
           'X': [],
           'Y': [],
           'class_name': [],
           'gene_name': ge.Genes.values.tolist(),
           'col': [],
           'GeneExp': np.empty([M, N], dtype=np.int32)
           }
    for i in range(N):
        # select a class
        bc = best_classes[i]
#         print(bc)

        # carve out data only relevant to the selected class
        class_df = df[df['best_class'] == bc]

        # randomly select a cell of that specific class
        cid = random.choice(class_df.index)
        temp = class_df.loc[cid]

        # keep the data for that particular cell to a dictionary
        out['cid'].append(cid)
        out['Cell_Num'].append(temp['Cell_Num'])  # This is 1-based, not 0-based
        out['X'].append(temp['X'])
        out['Y'].append(temp['Y'])
        out['class_name'].append(temp['best_class'])
#         start = time.time()
        mask = [i for i in range(len(class_list)) if class_list[i] == bc]
#         print(time.time() - start)
        col = random.choice(mask)
        out['col'].append(col)
        out['GeneExp'][:, i] = ge[:, col]

    return out

In [70]:
def thinner(data):
    '''
    shrink the gene expressions towards zero
    '''
    p = 0.1
    mat = data['GeneExp']
    rnd = np.random.binomial(mat, p)
    data['GeneExp'] = rnd
    return data

In [71]:
def position_genes(data):
    '''
    place the genes on the plane, centered around the corresponding cell center
    where the locations are distributed jointly normally (and independedly) with st dev r
    '''
    r = 8.8673
    u = np.random.normal(0, r, data['GeneExp'].shape)
    v = np.random.normal(0, r, data['GeneExp'].shape)
    _xCoord = (data["X"]+u)*(data['GeneExp'] > 0)
    _yCoord = (data["Y"]+v)*(data['GeneExp'] > 0)

    xCoord = pd.DataFrame(_xCoord,
                          columns=data['class_name'],
                          index=data['gene_name']
                          )
    yCoord = pd.DataFrame(_yCoord,
                          columns=data['class_name'],
                          index=data['gene_name']
                          )

    # print('in position')
    return xCoord, yCoord

In [72]:
def flatten(df):
    '''
    flattens a dataframe
    '''
    temp = df.unstack()
    values = temp.values
    labels = temp.index.droplevel().values
    out = np.hstack( (labels[:, None], values[:, None]) )
    return out

In [73]:
def xy_pairs(x, y):
    '''
    zips x, y lists to produce the coordinates
    '''

    # check is x and y are aligned
    assert np.all(x[:, 0] == y[:, 0])

    # remove if both are zero
    isZero = (np.array(x[:, 1]) == 0) & (np.array(y[:, 1]) == 0)
    x = x[~isZero, :]
    y = y[~isZero, :]

    # return the gene names and associated coordinates
    _labels = x[:, 0]
    _x = x[:, 1]
    _y = y[:, 1]
    out = np.hstack((_labels[:, None], _x[:, None], _y[:, None]))
    return out

In [74]:
def post_process(df1, df2):
    x = flatten(df1)
    y = flatten(df2)
    out = xy_pairs(x, y)
    return out

In [75]:
if __name__ == "__main__":
    n = 1000 #sample size
    xCoord = []
    yCoord = []

    # Fetch the data
    raw_data, gene_expression = fetch_data()

    # for each cell find its most likely cell class
    bc = best_class(raw_data)

    # stick it at the end of the dataframe
    raw_data['best_class'] = bc

    # remove cells belonging to the Zero class
    nonZero = raw_data['best_class'] != 'Zero'
    raw_data = raw_data[nonZero]

    spots = np.empty((0, 3))
    start = time.time()
    while spots.shape[0] < n:
        sample = draw_gene_expression(raw_data, gene_expression)

        sample = thinner(sample)

        _xCoord, _yCoord = position_genes(sample)
        _spots = post_process(_xCoord, _yCoord)
        spots = np.append(spots, _spots, axis=0)
        xCoord.append(_xCoord)
        yCoord.append(_yCoord)


    print('Done!')
    print('Collected %d spots in %4.2f secs' % (spots.shape[0], time.time()-start))


Done!
Collected 1197 spots in 0.26 secs


In [76]:
xCoord[0].shape

(92, 59)

In [77]:
xCoord[0].head()

,Astro.1,Astro.2,Astro.3,Astro.4,Astro.5,Cacna2d1.Lhx6.Reln,Cacna2d1.Lhx6.Vwa5a,Cacna2d1.Ndnf.Cxcl14,Cacna2d1.Ndnf.Npy,Cacna2d1.Ndnf.Rgs10,...,Sst.Erbb4.Crh,Sst.Erbb4.Rgs10,Sst.Npy.Cort,Sst.Npy.Mgat4c,Sst.Npy.Serpine2,Sst.Npy.Zbtb20,Sst.Pnoc.Calb1.Igfbp5,Sst.Pnoc.Calb1.Pvalb,Sst.Pnoc.Pvalb,Vip.Crh.C1ql1
3110035E14Rik,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,12621.733391,0.0,0.0,0.000000,0.0,0.000000,0.000000
6330403K07Rik,8002.295635,0.0,7607.675304,0.0,0.00000,8971.577508,0.0,0.0,0.0,0.0,...,11761.566168,8008.689148,9597.736602,12626.392559,0.0,0.0,12512.607311,0.0,0.000000,9344.427879
Adgrl2,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
Aldoc,0.000000,0.0,7614.806897,0.0,7776.91926,0.000000,0.0,0.0,0.0,0.0,...,0.000000,8016.788631,0.000000,0.000000,0.0,0.0,0.000000,0.0,12512.005484,0.000000
Arpp21,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [78]:
yCoord[0].head()

,Astro.1,Astro.2,Astro.3,Astro.4,Astro.5,Cacna2d1.Lhx6.Reln,Cacna2d1.Lhx6.Vwa5a,Cacna2d1.Ndnf.Cxcl14,Cacna2d1.Ndnf.Npy,Cacna2d1.Ndnf.Rgs10,...,Sst.Erbb4.Crh,Sst.Erbb4.Rgs10,Sst.Npy.Cort,Sst.Npy.Mgat4c,Sst.Npy.Serpine2,Sst.Npy.Zbtb20,Sst.Pnoc.Calb1.Igfbp5,Sst.Pnoc.Calb1.Pvalb,Sst.Pnoc.Pvalb,Vip.Crh.C1ql1
3110035E14Rik,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,17532.277199,0.0,0.0,0.000000,0.0,0.000000,0.000000
6330403K07Rik,15190.359093,0.0,15039.716534,0.0,0.000000,17236.063175,0.0,0.0,0.0,0.0,...,17941.141651,16489.398057,17373.696419,17532.237461,0.0,0.0,17576.787393,0.0,0.000000,16665.938109
Adgrl2,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000
Aldoc,0.000000,0.0,15049.198448,0.0,17025.596716,0.000000,0.0,0.0,0.0,0.0,...,0.000000,16512.875791,0.000000,0.000000,0.0,0.0,0.000000,0.0,17619.724074,0.000000
Arpp21,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000


In [79]:
sample['GeneExp'].shape

(92, 59)

In [80]:
spots

array([['6330403K07Rik', 8002.295634990178, 15190.359092852614],
       ['Id2', 8002.153868661382, 15208.648242059002],
       ['Kcnk2', 8003.979561947527, 15200.26485808963],
       ...,
       ['Synpr', 9318.70962466067, 16687.969598393098],
       ['Tac2', 9336.776682364709, 16675.19024092365],
       ['Vip', 9341.195163485798, 16678.83838752091]], dtype=object)